# The Importance of Normalizing Features for PCA 

In this notebook, I run through a practice example on the importance of feature scaling when planning to run a principal components procedure(PCA). This notebook assumes some basic understanding of what PCA and singular value decomposition. I quickly learned this lesson as a student working on a project for a class at Virginia Tech. 

I was working on a project related to opioid overdoses in the state of Ohio from 2011-2015. We were interested in exploring how differences in socioeconomic factors about certain counties related to the opioid usage in that county. To do this, my team planned to perform PCA on the socioeconomic data and perform k-means clustering on the resulting principal components. Then, we would do follow up analysis surrounding the opioid use within the naturally occurring clusters. PCA is useful in this scenario for two main reasons. The first is the socioeconomic data exhibits multicollinearity in our features. Secondly, PCA makes visualizing our Kmeans procedure easier. In this notebook, I present to you the data as I had it and how the failing to normalize my data affected my PCA which would of in turn affected analysis further downstream. So lets go to it..

In [ ]:
import pandas as pd
